In [ ]:
!pwd

In [1]:
import requests
import json
import popelines
import os
from datetime import datetime

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)


In [3]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

In [4]:
pope = popelines.popeline(dataset_id='vicky', service_key_file_loc=gbq_key, directory='.', verbose=False)


In [5]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

# helper flatten(jayson, {}, '')

In [6]:
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                for j in v:
                    flatten(j, acc, prefix_k)
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

# Testing kwargs

In [ ]:
def callback(arg_1, arg_2):
    print(arg_1)
    print(arg_2)

In [ ]:
def fix_json_keys(**kwargs):
    callback(**kwargs)
    

In [ ]:
fix_json_keys(arg_1='results', arg_2='experiments')

In [ ]:
j_ts

In [7]:
def fix_values(value, key, reset_key):
        if key == reset_key:
            new_list = []
            for x in value:
                value[x]['result_id'] = x
                new_list.append(value[x])
            return new_list
        else:
            return value


In [10]:
new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')


In [11]:
new_j_ts_list = [new_j_ts]

In [ ]:
new_j_ts_list

In [ ]:
pope.write_to_json(file_name='test_results_2.json', jayson=new_j_ts_list, mode='w')
    
    

In [ ]:
generate-schema --keep_nulls < test_results_2.json > test_schema.json

In [ ]:
pope.write_to_bq(table_name='result_ts', file_name='test_results_2.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


# Getting Results (from experiment)

In [ ]:
# get all projects
response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
j = json.loads(response_proj.text)
project_id = list()
for p in j:
    project_id.append(p['id'])  

# get all experiments from one project
p_id = project_id[0]  
params = (
    ('project_id', p_id),
    ('per_page', 100),
) 
response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
# j_exp contains all experiment id's
j_exp = json.loads(response_exp.text)

# get results from one experiment
# experiment_id = j_exp[0]['id']
# response_res = requests.get(f'https://api.optimizely.com/v2/experiments/{experiment_id}/results', headers=headers)



In [15]:
# get result time series from one experiment
experiment_id = 10815382176
response_ts = requests.get(f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries', headers=headers)


In [17]:
response_ts.text

''

In [16]:
j_ts = json.loads(response_ts.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
j_ts

In [ ]:
pp.pprint(j_ts)

In [ ]:
def fix_values(value, key):
    if key == 'results':
        new_list = []
        for x in value:
            value[x]['result_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value


In [ ]:
new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts)

In [ ]:
for key, val in new_j_ts.items():
    print(key, type(val))
    if isinstance(val, list):
#         print(f'list: {new_j_ts[key]}')
        print(f'list of length: {len(val)}, key is {key}')
    elif isinstance(val, dict):
#         print(f'dict: {new_j_ts[key]}')
        print(f'dict containing keys: {val.keys()}')
        for sub_key, sub_val in val.items():
            print(sub_key,type(sub_val))

In [ ]:
for k, v in new_j_ts['metrics'][0].items():
    print(k,type(v))
# pp.pprint(new_j_ts['metrics'][0])

In [ ]:
for k, v in new_j_ts['metrics'][0]['results'][0].items():
    print(k, type(v))

In [ ]:
new_j_ts['metrics'][0]['results'][0]['timeseries'][0]

## Trying to flatten inner level 'timeseries'

In [ ]:
flattened_timeseries = []
for element in new_j_ts['metrics'][0]['results'][0]['timeseries']:
    flattened_timeseries.append(flatten(element, {}, ''))
print(len(flattened_timeseries))

In [ ]:
type(flattened_timeseries)

## Replace old 'timeseries' with new 'flattened_timeseries'

In [ ]:
def populating_vals(outer_dict, inner_flattened_list, destination_key):
    updated_dict = {}
    for k, v in outer_dict.items():
        if k != destination_key:
            updated_dict[k] = v
        else:
            updated_dict[k] = inner_flattened_list
    return updated_dict

In [ ]:
def flatten_dupe_vals(vals, key):
    
    duped_results = []
    for element in vals[key]:
        ts_dict = populating_vals(outer_dict=vals, inner_flattened_list=element, destination_key=key)
        duped_results.append(ts_dict)

    flattened_results = []
    for element in duped_results:
        flattened_results.append(flatten(element, {}, ''))
        
    return flattened_results

In [ ]:
updated_results = populating_vals(outer_dict=new_j_ts['metrics'][0]['results'][0], inner_flattened_list=flattened_timeseries, destination_key='timeseries')
flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')




In [ ]:
update_metrics = populating_vals(outer_dict=new_j_ts['metrics'][0], inner_flattened_list=flattened_results, destination_key='results')
flattened_metrics = flatten_dupe_vals(vals=update_metrics, key='results')


In [ ]:
flattened_metrics

In [ ]:
json.dump(new_j_ts, open('somejson.json', 'w'))

In [ ]:
metric = new_j_ts['metrics'][0]
ts = metric['results'][0]

In [ ]:
flattened_metrics = []
for metric in new_j_ts['metrics']:
    for ts in metric['results']:
        flattened_timeseries = []
        for element in ts['timeseries']:
            flattened_timeseries.append(flatten(element, {}, ''))

        # Replace old 'timeseries' with new 'flattened_timeseries'
        updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
        flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

    # Replace old 'metrics' with new 'flattened_results'
    update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
    flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
    

In [ ]:
flattened_metrics

In [ ]:
flattened_metrics[0]

# Testing main.py project-experiment

In [ ]:
import requests
import json
import popelines
import os
from datetime import datetime


# a function to detect deeply nested json file, with a mix of dictionaries and lists
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                for j in v:
                    flatten(j, acc, prefix_k)
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [ ]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [ ]:
# Upload Project-Experiment table
# get all projects
try:
    response_proj = requests.get('https://api.optimizely.com/v2/projects', headers=headers)
    j_proj = json.loads(response_proj.text)
    response_proj.raise_for_status()
    print('Successfully read project data.')

except requests.exceptions.HTTPError as err:
    print(err)

In [ ]:
# store a list of project metadata
project_list = list()

for project in j_proj:
    p_id = project['id']

    # get all experiments from one project
    params = (
        ('project_id', p_id),
        ('per_page', 100),
    ) 

    try:
        response_exp = requests.get('https://api.optimizely.com/v2/experiments', headers=headers, params=params)
        j_exp = json.loads(response_exp.text)
        response_exp.raise_for_status()
        print('Successfully read experiment data.')
    except requests.exceptions.HTTPError as err:
        print(err)

    # loop for all experiments in this project
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = p_id
        upload_exp_list.append(flatten(exp, {}, ''))

    # upload experiment 
    pope.write_to_json(file_name='experiments.json', jayson=upload_exp_list, mode='w')
    pope.write_to_bq(table_name='experiments', file_name='experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    print(f"Successfully uploaded experiment {exp['id']}")

    project_list.append(project)

In [ ]:
project_list

# Testing main.py experiment-result-timeseries